# **Projet de Sécurité et protection de l’information médicale - BIO-5103A**
## Hippolyte PASCAL - E5 BIO

# **Sources IA**

* [Retravailler sur ma première esquisse de code de chiffrement basé sur les TP](https://chatgpt.com/share/672e5024-c8dc-800f-9690-de887107612c)
* [Pour calculer l'age en mois](https://chatgpt.com/share/672e5024-c8dc-800f-9690-de887107612c)
* [Pour générer les listes de mots sensibles dans la déidentification textuelle](https://chatgpt.com/share/672e5179-a404-8010-93af-4947647899b0)
* Clarification et détail des commentaires avec l'IA


# **Déidentification de données de santé structurées**

In [26]:
import pandas as pd
import random
import sympy  
import hashlib # Librairie pour les fonctions de hachage cryptographique
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes  # Librairies pour les algorithmes de chiffrement symétrique
from cryptography.hazmat.primitives.asymmetric import rsa  # Librairie pour le chiffrement asymétrique RSA
from cryptography.hazmat.primitives import serialization  # Librairie pour la sérialisation des clés
from cryptography.hazmat.primitives import hashes  # Librairie pour les fonctions de hachage
from cryptography.hazmat.primitives.asymmetric import padding as rsa_padding  # Librairie pour le padding RSA
from cryptography.hazmat.backends import default_backend  # Backend cryptographique par défaut
from cryptography.hazmat.primitives import padding  # Librairie pour le padding des messages
import os  # Librairie pour interagir avec le système d'exploitation
import binascii  # Librairie pour la manipulation de données binaires
from dateutil.relativedelta import relativedelta  # Librairie pour la manipulation des dates et durées (par exemple, ajout de mois ou d'années)
import re

In [16]:
# Load dataset
data = pd.read_excel("Data_id.xlsx", sheet_name="data_PASCAL_Hippolyte")

In [17]:
# Générer un nombre premier ou non-premier en fonction du sexe
def generate_random_prime_or_non_prime(sexe):
    if sexe == "H":
        # Générer un nombre premier entre 2 et 100
        return sympy.randprime(2, 100)
    else:
        # Générer un nombre non-premier entre 2 et 100
        while True:
            num = random.randint(2, 100)
            if not sympy.isprime(num):
                return num


In [18]:
# Remplacer directement les valeurs de la colonne "sexe" par des nombres premiers ou non-premiers
if 'sexe' in data.columns:
    data['sexe'] = data['sexe'].apply(generate_random_prime_or_non_prime)

# Transformer "sexe médecin" en nombre premier ou non-premier
if 'sexe médecin' in data.columns:
    data['sexe médecin'] = data['sexe médecin'].apply(generate_random_prime_or_non_prime)


In [19]:
# Fonction de hachage avec sel (non réversible)
def hash_sha256_with_salt(value, salt):
    if pd.isnull(value):  # Gérer les valeurs NaN
        return None
    return hashlib.sha256((str(value) + salt).encode()).hexdigest()

In [20]:
# Fonction de chiffrement AES (réversible)
def encrypt_aes(value, key, mode):
    if pd.isnull(value):  # Gérer les valeurs NaN
        return None
    if mode == 'CBC':
        iv = os.urandom(16)  # Initialisation aléatoire pour le mode CBC
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    elif mode == 'ECB':
        cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    encryptor = cipher.encryptor()
    padder = padding.PKCS7(128).padder()  # Applique un remplissage PKCS7
    padded_data = padder.update(str(value).encode()) + padder.finalize()  # Ajoute le remplissage aux données
    if mode == 'CBC':
        return iv + encryptor.update(padded_data) + encryptor.finalize()  # Retourne les données chiffrées avec l'IV pour CBC
    elif mode == 'ECB':
        return encryptor.update(padded_data) + encryptor.finalize()  # Retourne les données chiffrées pour ECB

In [21]:
# Générer la clé AES et le sel
aes_key = os.urandom(32)  # Génère une clé AES aléatoire de 32 octets
salt = binascii.hexlify(os.urandom(16)).decode()  # Génère un sel aléatoire de 16 octets et le convertit en hexadécimal

# Générer une paire de clés RSA
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)
public_key = private_key.public_key()  # Récupère la clé publique à partir de la clé privée

# Chiffrer la clé AES et le sel avec RSA
encrypted_aes_key = public_key.encrypt(
    aes_key,
    rsa_padding.OAEP(
        mgf=rsa_padding.MGF1(algorithm=hashes.SHA256()),  # Utilisation de la fonction de génération de masque MGF1 avec SHA256
        algorithm=hashes.SHA256(),  # Utilisation de l'algorithme de hachage SHA256
        label=None
    )
)
encrypted_salt = public_key.encrypt(
    salt.encode(),  # Encode le sel en bytes avant le chiffrement
    rsa_padding.OAEP(
        mgf=rsa_padding.MGF1(algorithm=hashes.SHA256()),  # Même fonction de génération de masque MGF1 avec SHA256
        algorithm=hashes.SHA256(),  # Utilisation de l'algorithme de hachage SHA256
        label=None
    )
)

# Sauvegarder la clé AES chiffrée et le sel dans un fichier
with open("encrypted_keys.txt", "wb") as f:
    f.write(encrypted_aes_key)  # Écriture de la clé AES chiffrée dans le fichier
    f.write(encrypted_salt)  # Écriture du sel chiffré dans le fichier


In [22]:
# Fonction pour créer un pseudo et le chiffrer
def create_and_encrypt_pseudo(row, fields, mode):
    pseudo = '_'.join(str(row[field]) for field in fields)  # Création du pseudo en concatenant les valeurs des champs
    encrypted_pseudo = encrypt_aes(pseudo, aes_key, mode)  # Chiffrement du pseudo avec AES
    return encrypted_pseudo

# Création des pseudos et chiffrement
data['encrypted_pseudo_patient_hôpital'] = data.apply(lambda row: create_and_encrypt_pseudo(row, ['nom patient', 'prénom patient', 'sexe', 'date de naissance patient'], 'CBC'), axis=1)
data['encrypted_pseudo_patient_addresse'] = data.apply(lambda row: create_and_encrypt_pseudo(row, ['adresse patient', 'code postal patient', 'Code INSEE commune patient', 'Code INSEE Arrondissement', 'Code INSEE canton'], 'ECB'), axis=1)
data['encrypted_pseudo_social_security_number'] = data.apply(lambda row: create_and_encrypt_pseudo(row, ['Numéro de sécurité sociale'], 'CBC'), axis=1)
data['encrypted_pseudo_health_identifier'] = data.apply(lambda row: create_and_encrypt_pseudo(row, ['Identifiant national de santé'], 'CBC'), axis=1)

# Hachage des pseudos et autres informations sensibles
data['hashed_encrypted_pseudo_patient_hôpital'] = data['encrypted_pseudo_patient_hôpital'].apply(lambda x: hash_sha256_with_salt(x, salt))
data['hashed_encrypted_pseudo_patient_addresse'] = data['encrypted_pseudo_patient_addresse'].apply(lambda x: hash_sha256_with_salt(x, salt))
data['hashed_encrypted_social_security_number'] = data['encrypted_pseudo_social_security_number'].apply(lambda x: hash_sha256_with_salt(x, salt))
data['hashed_encrypted_health_identifier'] = data['encrypted_pseudo_health_identifier'].apply(lambda x: hash_sha256_with_salt(x, salt))

# Suppression des colonnes sensibles
columns_to_drop = [
    'Numéro de sécurité sociale', 'Identifiant national de santé',
    'nom patient', 'prénom patient', 'sexe', 'date de naissance patient',
    'adresse patient', 'code postal patient', 'Code INSEE commune patient',
    'Code INSEE Arrondissement', 'Code INSEE canton', 'Code INSEE Département',
    'Nom de commune patient majuscule', 'Nom de commune patient minuscule', 'Libellé commune patient',
    'encrypted_pseudo_patient_hôpital','encrypted_pseudo_patient_addresse','encrypted_pseudo_social_security_number','encrypted_pseudo_health_identifier'
]

pseudo_data = data.drop(columns=columns_to_drop)  # Suppression des colonnes sensibles du dataset

# Sauvegarde du fichier pseudonymisé
pseudo_data.to_csv("data_pseudo_hosto.csv", index=False)

# Création des pseudos et chiffrement pour les informations des médecins et des diagnostics
data['encrypted_pseudo_médecin_info'] = data.apply(lambda row: create_and_encrypt_pseudo(row, ['titre médecin', 'sexe médecin', 'nom médecin', 'prénom médecin', 'spécialité', 'code FINESS Etablissement de Santé du médecin'], 'ECB'), axis=1)
data['encrypted_pseudo_diag_1'] = data.apply(lambda row: create_and_encrypt_pseudo(row, ['Code CIM-10 Diagnostic 1'], 'CBC'), axis=1)
data['encrypted_pseudo_diag_2'] = data.apply(lambda row: create_and_encrypt_pseudo(row, ['Code CIM-10 Diagnostic 2'], 'ECB'), axis=1)
data['encrypted_pseudo_intervention_1'] = data.apply(lambda row: create_and_encrypt_pseudo(row, ['Code CCAM Acte 1'], 'ECB'), axis=1)
data['encrypted_pseudo_intervention_2'] = data.apply(lambda row: create_and_encrypt_pseudo(row, ['Code CCAM Acte 2'], 'ECB'), axis=1)
data['encrypted_pseudo_intervention_3'] = data.apply(lambda row: create_and_encrypt_pseudo(row, ['Code CCAM Acte 3'], 'ECB'), axis=1)

# Créer des pseudos et les chiffrer pour les interventions et diagnostics (jusqu'à 5)
for i in range(1, 6):
    data[f'encrypted_pseudo_intervention_{i}'] = data.apply(lambda row: create_and_encrypt_pseudo(row, [f'Date AMMM Traitement {i}', f'Code CIS Traitement {i}'], 'CBC'), axis=1)


In [23]:
# Fonction pour calculer l'âge en mois
def calculate_age_in_months(birth_date, consultation_date):
    if pd.isnull(birth_date) or pd.isnull(consultation_date):
        return None
    diff = relativedelta(consultation_date, birth_date)  # Calcul de la différence entre les deux dates
    return round(diff.years * 12 + diff.months + diff.days / 30.44, 2)  # Retourne l'âge en mois, arrondi à deux décimales

# S'assurer que les dates sont au format datetime
data['date de naissance patient'] = pd.to_datetime(data['date de naissance patient'], errors='coerce')

# Colonnes de consultation
consultation_columns = [col for col in data.columns if col.startswith("Date consultation")]

# Calculer l'âge en mois pour chaque consultation
for col in consultation_columns:
    data[col] = pd.to_datetime(data[col], errors='coerce')  # Conversion des dates de consultation en datetime
    age_column = f' {col}'  # Créer le nom de la nouvelle colonne pour l'âge
    data[age_column] = data.apply(lambda row: calculate_age_in_months(row['date de naissance patient'], row[col]), axis=1)  # Appliquer la fonction sur chaque ligne


In [24]:
# Liste des colonnes sensibles à supprimer
columns_to_drop = [
    # Informations sur le médecin
    'titre médecin', 'sexe médecin', 'nom médecin', 'prénom médecin', 'spécialité',
    'code FINESS Etablissement de Santé du médecin', 'Adresse établissement de santé du médecin',
    
    # Codes et libellés diagnostiques
    'Code CIM-10 Diagnostic 1', 'Code CIM-10 Diagnostic 2',
    'Code CCAM Acte 1', 'Code CCAM Acte 2', 'Code CCAM Acte 3', 
    'Libellé CCAM Acte 1', 'Libellé CCAM Acte 2', 'Libellé CCAM Acte 3',
    'Libellé CIM-10 Diagnostic 1', 'Libellé CIM-10 Diagnostic 2',
]

# Suppression des colonnes sensibles pour Code CCAM Acte, Date AMMM Traitement, Code CIS Traitement, et Code CIM-10 Diagnostic jusqu'à 5
for i in range(1, 6):
    columns_to_drop.extend([ f'Date AMMM Traitement {i}', f'Code CIS Traitement {i}', f'Dénomination Traitement {i}'])

# Suppression des colonnes liées aux consultations
columns_to_drop.extend(consultation_columns)

# Suppression des colonnes sensibles du DataFrame
final_data = data.drop(columns=columns_to_drop)

# Sauvegarde des données anonymisées dans un fichier CSV
final_data.to_csv("data_pseudo_stat.csv", index=False)


# **Déidentification textuelle**

In [27]:
# Fonction pour charger une liste de mots depuis un fichier
def charger_mots(fichier):
    with open(fichier, 'r', encoding='utf-8') as f:
        return [ligne.strip() for ligne in f if ligne.strip()]

# Charger les listes de mots sensibles depuis des fichiers
noms_prénoms = charger_mots("noms_prénoms.txt")  # Liste des noms et prénoms sensibles
adresses = charger_mots("adresses.txt")        # Liste des adresses sensibles
données_médicales = charger_mots("données_médicales.txt")  # Liste des termes médicaux sensibles
dates = charger_mots("dates.txt")  # Liste des dates sensibles (si tu en as une)

# Listes de remplacement aléatoires crédibles
noms_fictifs = ["Pierre Dupont", "Marie Durand", "Jacques Martin", "Sophie Lefevre", "Lucie Bernard"]
adresses_fictives = ["10 rue des Fleurs, Paris", "25 avenue de la République, Lyon", "12 boulevard Saint-Germain, Paris", "33 rue de la Liberté, Marseille"]
termes_médicaux = ["infection virale", "fièvre élevée", "fracture du tibia", "hypertension", "diabète de type 2"]

# Liste de dates fictives
dates_fictives = ["15/03/2010", "21/07/2012", "03/12/2015", "10/05/2018", "25/09/2020", "18/01/2021", "29/06/2022", "05/11/2023", "12/04/2024", "07/08/2025"]

# Fonction de remplacement
def remplacer_mots(texte, nom_prenom, adresse, donnée_médicale, dates):
    def remplacement_aleatoire(match):
        mot_trouvé = match.group(0)
        
        # Remplacer par un nom aléatoire si le mot est dans la liste des noms/prénoms
        if mot_trouvé.lower() in [nom.lower() for nom in nom_prenom]:
            return random.choice(noms_fictifs)
        
        # Remplacer par une adresse aléatoire si le mot est dans la liste des adresses
        elif mot_trouvé.lower() in [adresse.lower() for adresse in adresse]:
            return random.choice(adresses_fictives)
        
        # Remplacer par un terme médical aléatoire si le mot est dans la liste des données médicales
        elif mot_trouvé.lower() in [donnée.lower() for donnée in donnée_médicale]:
            return random.choice(termes_médicaux)

        # Remplacer par une date aléatoire si le mot ressemble à une date (format dd/mm/yyyy ou yyyy-mm-dd ou jour mois année)
        elif re.match(r'\b(\d{2}/\d{2}/\d{4}|\d{4}-\d{2}-\d{2}|\d{2} \w+ \d{4})\b', mot_trouvé):  # Formats supportés
            return random.choice(dates_fictives)
        
        # Si le mot ne correspond à aucune catégorie, ne pas le changer
        else:
            return mot_trouvé
    
    # Construire les patterns regex pour détecter les mots dans chaque catégorie
    pattern_nom_prenom = r'\b(' + '|'.join(re.escape(mot) for mot in nom_prenom) + r')\b'
    pattern_adresse = r'\b(' + '|'.join(re.escape(mot) for mot in adresse) + r')\b'
    pattern_donnée_médicale = r'\b(' + '|'.join(re.escape(mot) for mot in donnée_médicale) + r')\b'
    pattern_dates = r'\b(\d{2}/\d{2}/\d{4}|\d{4}-\d{2}-\d{2}|\d{2} \w+ \d{4})\b'  # Regex pour les dates (formats dd/mm/yyyy, yyyy-mm-dd, jour mois année)
    
    # Remplacer chaque catégorie dans le texte
    texte_remplacé = re.sub(pattern_nom_prenom, remplacement_aleatoire, texte, flags=re.IGNORECASE)
    texte_remplacé = re.sub(pattern_adresse, remplacement_aleatoire, texte_remplacé, flags=re.IGNORECASE)
    texte_remplacé = re.sub(pattern_donnée_médicale, remplacement_aleatoire, texte_remplacé, flags=re.IGNORECASE)
    texte_remplacé = re.sub(pattern_dates, remplacement_aleatoire, texte_remplacé)  # Appliquer la substitution pour les dates
    
    return texte_remplacé

# Dossier contenant les rapports médicaux
dossier_rapports = "Rapports_médicaux"

# Vérifie si le dossier existe
if not os.path.exists(dossier_rapports):
    print(f"Erreur : Le dossier '{dossier_rapports}' n'existe pas.")
else:
    # Parcourir chaque fichier texte dans le dossier
    for nom_fichier in os.listdir(dossier_rapports):
        chemin_fichier = os.path.join(dossier_rapports, nom_fichier)
        
        # Vérifier que c'est bien un fichier texte et non un dossier
        if os.path.isfile(chemin_fichier) and nom_fichier.endswith(".txt"):
            # Lire le contenu du fichier
            with open(chemin_fichier, 'r', encoding='utf-8') as fichier:
                texte = fichier.read()

            # Appliquer la fonction de remplacement
            texte_anonymisé = remplacer_mots(texte, noms_prénoms, adresses, données_médicales, dates)
            
            # Sauvegarder le texte anonymisé dans un nouveau fichier
            chemin_fichier_anonymisé = os.path.join(dossier_rapports, "anonymisé_" + nom_fichier)
            with open(chemin_fichier_anonymisé, 'w', encoding='utf-8') as fichier_anonymisé:
                fichier_anonymisé.write(texte_anonymisé)

            print(f"\nLe fichier {nom_fichier} a été anonymisé avec succès.")



Le fichier CRH.txt a été anonymisé avec succès.

Le fichier CRO.txt a été anonymisé avec succès.

Le fichier Rapport 1.txt a été anonymisé avec succès.

Le fichier Rapport 2.txt a été anonymisé avec succès.

Le fichier Rapport 3.txt a été anonymisé avec succès.
